In [0]:
# Lets check the available gpu in colab
!nvidia-smi

In [0]:
# For this notebook we are going to use keras - retinanet 
# Lets install Keras retina-net for the same
!git clone https://github.com/fizyr/keras-retinanet.git

!pip install --upgrade keras

%cd keras-retinanet/
!pip install .

!python setup.py build_ext --inplace

!pip install gdown
!pip install tensorflow-gpu

In [0]:
# Mount google drive so that we can read data from the drive
from google.colab import drive
drive.mount('/content/drive')
'''
# Unmount Drive 
from google.colab import drive
drive.flush_and_unmount()
'''

In [0]:
# Lets load the required packes required for this problem
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from pandas.plotting import register_matplotlib_converters
from sklearn.model_selection import train_test_split
import urllib
import os
import csv
import cv2
import time
from PIL import Image

from keras_retinanet import models
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.visualization import draw_box, draw_caption
from keras_retinanet.utils.colors import label_color


# Lets set some parameters required
%matplotlib inline
%config InlineBackend.figure_format='retina'

register_matplotlib_converters()
sns.set(style='whitegrid', palette='muted', font_scale=1.5)

rcParams['figure.figsize'] = 22, 10

RANDOM_SEED = 42

np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

In [0]:
# below are the location of the files
root_dir = '/content/drive/My Drive/global-wheat-detection'
train_dir = os.path.join(root_dir,'train')
test_dir = os.path.join(root_dir,'test')

In [0]:
# Lets check the directory and files present

for filefolder in os.listdir(root_dir):
  print(os.path.join(root_dir,filefolder))

In [0]:
# Lets read the data and also submission file to understand the data
train_df = pd.read_csv('/content/drive/My Drive/global-wheat-detection/train.csv')
sample_submit = pd.read_csv('/content/drive/My Drive/global-wheat-detection/sample_submission.csv')

#### Below are some basic checkes before starting the problem

In [0]:
sample_submit.tail()

In [0]:
train_df_master = train_df.copy()

In [0]:
train_df.columns

Index(['image_id', 'width', 'height', 'bbox', 'source'], dtype='object')

In [0]:
train_df.shape

(147793, 5)

In [0]:
train_df.dtypes

In [0]:
# We can see from the min and max values all the images are of 1024x1024
train_df.describe()

In [0]:
train_df.isnull().sum()

image_id    0
width       0
height      0
bbox        0
source      0
dtype: int64

In [0]:
train_df.head()

,image_id,width,height,bbox,source
0,b6ab77fd7,1024,1024,"[834.0, 222.0, 56.0, 36.0]",usask_1
1,b6ab77fd7,1024,1024,"[226.0, 548.0, 130.0, 58.0]",usask_1
2,b6ab77fd7,1024,1024,"[377.0, 504.0, 74.0, 160.0]",usask_1
3,b6ab77fd7,1024,1024,"[834.0, 95.0, 109.0, 107.0]",usask_1
4,b6ab77fd7,1024,1024,"[26.0, 144.0, 124.0, 117.0]",usask_1


In [0]:
# Below are the distribution of labels in the dataset 
plt.figure(figsize=(10,5))
sns.countplot('source',data=train_df)
plt.show()

In [0]:
# Remove braces from bbox
train_df['bbox'] = train_df.bbox.str.replace('[','')
train_df['bbox'] = train_df.bbox.str.replace(']','')

In [0]:
# Creating different coordinates require for image localization
train_df['x'] = train_df.bbox.str.split(',').str[0].astype(np.float)
train_df['y'] = train_df.bbox.str.split(',').str[1].astype(np.float)
train_df['w'] = train_df.bbox.str.split(',').str[2].astype(np.float)
train_df['h'] = train_df.bbox.str.split(',').str[3].astype(np.float)

In [0]:
train_df['x'] = train_df['x'].astype(int)
train_df['y'] = train_df['y'].astype(int)
train_df['w'] = train_df['w'].astype(int)
train_df['h'] = train_df['h'].astype(int)

In [0]:
train_df.head()

,image_id,width,height,bbox,source,x,y,w,h
0,b6ab77fd7,1024,1024,"834.0, 222.0, 56.0, 36.0",usask_1,834,222,56,36
1,b6ab77fd7,1024,1024,"226.0, 548.0, 130.0, 58.0",usask_1,226,548,130,58
2,b6ab77fd7,1024,1024,"377.0, 504.0, 74.0, 160.0",usask_1,377,504,74,160
3,b6ab77fd7,1024,1024,"834.0, 95.0, 109.0, 107.0",usask_1,834,95,109,107
4,b6ab77fd7,1024,1024,"26.0, 144.0, 124.0, 117.0",usask_1,26,144,124,117


In [0]:
# Below function will show the actual images
def view_actual_image(image_id):
  train_path = os.path.join(root_dir,'train')
  img = cv2.imread(train_path+'/{}.jpg'.format(image_id))
  img = cv2.cvtColor(img, cv2.cv2.IMREAD_COLOR)
  plt.figure(figsize=(8,8))
  plt.imshow(img)

In [0]:
view_actual_image('b6ab77fd7')

In [0]:
# This finction will help to draw bounding box in actual image

def view_bb_image(row_data):
  train_path = os.path.join(root_dir,'train')
  img = cv2.imread(train_path+'/{}.jpg'.format(row_data.iloc[0].values[0]))
  # img = cv2.resize(img,(224,224))
  img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  for i,record in row_data.iterrows():
    cv2.rectangle(img,(int(record['x']), int(record['y'])),
                  (int(record['x']) + int(record['w']), 
                   int(record['y']) + int(record['h'])),
                  (220, 0, 0), 3)
  plt.figure(figsize=(8,8))
  plt.imshow(img)

In [0]:
view_bb_image(train_df[train_df.image_id=='b6ab77fd7'])

In [0]:
view_bb_image(train_df[train_df.image_id=='ffdf83e42'])

In [0]:
#Rename the columns
train_df_required = train_df[['image_id','x','y','w','h']]

In [0]:
train_df_required['class_name'] = 'wheat_head'

In [0]:
train_df_required['w'] = train_df_required.x + train_df_required.w
train_df_required['h'] = train_df_required.y + train_df_required.h

In [0]:
# As this algorithm donot expect w & h so better convert to (X_max,Y_max) this is done in the above step
train_df_required.columns = ['image_name','x_min','y_min','x_max','y_max','class_name']

In [0]:
train_df_required.head()

In [0]:
train_dir

'/content/drive/My Drive/global-wheat-detection/train'

### Below teps are required preprocessing step as per model documentation

In [0]:
train_df_required['image_name'] = train_dir+'/'+train_df_required['image_name']+'.jpg' 

In [0]:
ANNOTATIONS_FILE = 'annotations.csv'
CLASSES_FILE = 'classes.csv'

In [0]:
train_df_required.to_csv(ANNOTATIONS_FILE, index=False, header=None)

In [0]:
classes = set(['wheat_head'])

with open(CLASSES_FILE, 'w') as f:
  for i, line in enumerate(sorted(classes)):
    f.write('{},{}\n'.format(line,i))

In [0]:
!head classes.csv

wheat_head,0


In [0]:
!head annotations.csv

In [0]:
# We will use pretrained model as a base model for trainng
PRETRAINED_MODEL = './snapshots/_pretrained_model.h5'

URL_MODEL = 'https://github.com/fizyr/keras-retinanet/releases/download/0.5.1/resnet50_coco_best_v2.1.0.h5'
urllib.request.urlretrieve(URL_MODEL, PRETRAINED_MODEL)

print('Downloaded pretrained model to ' + PRETRAINED_MODEL)

Downloaded pretrained model to ./snapshots/_pretrained_model.h5


In [0]:
# This is model training step be careful with the number of epochs , It takes a huge time to tain
!keras_retinanet/bin/train.py \
  --freeze-backbone \
  --random-transform \
  --weights {PRETRAINED_MODEL} \
  --batch-size 8 \
  --steps 500 \
  --epochs 5 csv annotations.csv classes.csv

In [0]:
!ls snapshots

_pretrained_model.h5


In [0]:
# Loading the model
model_path = os.path.join('snapshots', sorted(os.listdir('snapshots'), reverse=True)[0])
# model_path = '/content/drive/My Drive/global-wheat-detection/resnet50_csv_05.h5'
print(model_path)

model = models.load_model(model_path, backbone_name='resnet50')
model = models.convert_model(model)

labels_to_names = pd.read_csv(CLASSES_FILE, header=None).T.loc[0].to_dict()

In [0]:
# Creating a test set for passing it to model consist of path of the images 
test_df = []
for img in os.listdir(test_dir):
  test_df.append(os.path.join(test_dir,img))

In [0]:
test_df = pd.DataFrame(test_df)

In [0]:
test_df.columns=['image_name']

#### Below function are required for the image prediction

In [0]:
def predict(image):
  image = preprocess_image(image.copy())
  image, scale = resize_image(image)

  boxes, scores, labels = model.predict_on_batch(
    np.expand_dims(image, axis=0)
  )

  boxes /= scale

  return boxes, scores, labels

In [0]:
THRES_SCORE = 0.6
def draw_detections(image, boxes, scores, labels):
  for box, score, label in zip(boxes[0], scores[0], labels[0]):
    if score < THRES_SCORE:
        break

    color = label_color(label)

    b = box.astype(int)
    draw_box(image, b, color=color)

    caption = "{} {:.3f}".format(labels_to_names[label], score)
    draw_caption(image, b, caption)

In [0]:
output = []
def show_detected_objects(image_row):
  img_path = image_row.image_name
  
  image = read_image_bgr(img_path)

  boxes, scores, labels = predict(image)
  # output.append(boxes)
  # return(output)


  draw = image.copy()
  draw = cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)

  draw_detections(draw, boxes, scores, labels)

  plt.axis('off')
  plt.imshow(draw)
  plt.show()

In [0]:
# Lets see how our model is predicting
for i in range(0,len(test_df)):
  show_detected_objects(test_df.iloc[i])

In [0]:
# Below are steps to be followed to get the BBOX Coordinates
preds=[]
imgid=[]
for img in os.listdir(test_dir):
    image = read_image_bgr(os.path.join(test_dir,img))
    boxes, scores, labels = predict(image)
    boxes=boxes[0]
    scores=scores[0]
    for idx in range(boxes.shape[0]):
        if scores[idx]>THRES_SCORE:
            box,score=boxes[idx],scores[idx]
            imgid.append(img.split(".")[0])
            preds.append("{} {} {} {} {}".format(score, int(box[0]), int(box[1]), int(box[2]-box[0]), int(box[3]-box[1])))

In [0]:
sub={"image_id":imgid, "PredictionString":preds}
sub=pd.DataFrame(sub)
sub.head()

,image_id,PredictionString
0,348a992bb,0.8609244227409363 914 565 84 91
1,348a992bb,0.8341029286384583 853 851 79 71
2,348a992bb,0.8312199115753174 934 786 87 83
3,348a992bb,0.8235329389572144 278 334 88 90
4,348a992bb,0.8039975166320801 145 39 105 90


In [0]:
sub_=sub.groupby(["image_id"])['PredictionString'].apply(lambda x: ' '.join(x)).reset_index()
sub_

In [0]:
for idx,imgid in enumerate(sample_submit['image_id']):
    sample_submit.iloc[idx,1]=sub_[sub_['image_id']==imgid].values[0,1]
    
sample_submit.head()

In [0]:
sample_submit.to_csv('/content/drive/My Drive/global-wheat-detection/submission.csv',index=False)